In [49]:
# import necessary packages
import numpy as np
import pandas as pd
import os

import allensdk
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache
from visual_behavior.data_access import loading
import visual_behavior.database as db
from multiprocessing import Pool

In [6]:
data_storage_directory = '/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/production_cache'

cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=data_storage_directory)

ophys_session_table.csv: 100%|██████████| 165k/165k [00:00<00:00, 1.27MMB/s] 
behavior_session_table.csv: 100%|██████████| 885k/885k [00:00<00:00, 3.12MMB/s] 
ophys_experiment_table.csv: 100%|██████████| 336k/336k [00:00<00:00, 1.44MMB/s] 


In [13]:
experiment_table = cache.get_ophys_experiment_table().reset_index()
len(experiment_table)

1165

In [14]:
experiment_table.sample(5)

,ophys_experiment_id,equipment_name,full_genotype,mouse_id,reporter_line,driver_line,sex,age_in_days,session_type,cre_line,...,prior_exposures_to_image_set,prior_exposures_to_omissions,ophys_session_id,behavior_session_id,ophys_container_id,project_code,imaging_depth,targeted_structure,date_of_acquisition,file_id
534,853362775,MESO.1,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,438912,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],M,136.0,OPHYS_5_images_B_passive,Vip-IRES-Cre,...,2.0,5,853177377,853266283,1018027862,VisualBehaviorMultiscope,150,VISl,2019-04-17 13:18:44.790144,1086012643
956,873963889,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,448366,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],M,127.0,OPHYS_5_images_B_passive,Sst-IRES-Cre,...,1.0,7,873720614,873813922,1018028429,VisualBehaviorMultiscope,225,VISp,2019-05-22 13:10:18.000000,1086012448
663,949723713,MESO.1,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,453991,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],M,222.0,OPHYS_5_images_B_passive,Vip-IRES-Cre,...,1.0,8,949217880,949348713,1022731527,VisualBehaviorMultiscope,150,VISp,2019-09-18 10:41:11.695425,1085400416
1093,853988454,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,440631,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],M,128.0,OPHYS_5_images_B_passive,Sst-IRES-Cre,...,1.0,5,853416532,853505213,1018028147,VisualBehaviorMultiscope,75,VISl,2019-04-18 08:47:33.972217,1085396727
135,990400775,MESO.1,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,479839,Ai93(TITL-GCaMP6f),"[Slc17a7-IRES2-Cre, Camk2a-tTA]",M,164.0,OPHYS_4_images_B,Slc17a7-IRES2-Cre,...,0.0,3,990139534,990193421,1018028049,VisualBehaviorMultiscope,165,VISp,2019-11-25 08:08:08.354949,1085399929


In [53]:
experiment_table['reporter_line'].unique()

array(['Ai148(TIT2L-GC6f-ICL-tTA2)', 'Ai94(TITL-GCaMP6s)',
       'Ai93(TITL-GCaMP6f)'], dtype=object)

In [50]:
def log_basic_cell_information(oeid):
    
    dataset = loading.get_ophys_dataset(oeid)


    conn = db.Database('visual_behavior_data')
    coll = conn['ophys_analysis']['cell_specimen_table']
    
    valid_cell_table = dataset.cell_specimen_table.query('valid_roi').reset_index()
    print('logging {} cells for oeid {}'.format(len(valid_cell_table), oeid))
    for idx, row in valid_cell_table.iterrows():
        cell_dict = {'ophys_experiment_id': oeid, 'ophys_session_id': dataset.ophys_session_id}
        cell_dict.update(row.to_dict())
        entry = db.clean_and_timestamp(cell_dict)
        db.update_or_create(coll, entry, keys_to_check = ['ophys_experiment_id', 'cell_specimen_id'])

    conn.close()

In [52]:
with Pool(32) as pool:
    pool.map(log_basic_cell_information, experiment_table['ophys_experiment_id'].unique())

logging 13 cells for oeid 995280513
logging 13 cells for oeid 953659743logging 7 cells for oeid 960410026
logging 8 cells for oeid 958741232

logging 17 cells for oeid 956941844
logging 17 cells for oeid 939471245
logging 22 cells for oeid 830093338
logging 12 cells for oeid 951980471
logging 17 cells for oeid 972200901
logging 10 cells for oeid 948704866
logging 20 cells for oeid 1048483611logging 16 cells for oeid 1050762974

logging 29 cells for oeid 989610991logging 59 cells for oeid 943151552

logging 65 cells for oeid 871155338
logging 31 cells for oeid 989213060
logging 85 cells for oeid 986518889
logging 73 cells for oeid 991852000
logging 117 cells for oeid 923100531
logging 17 cells for oeid 950833329
logging 5 cells for oeid 1050485657
logging 10 cells for oeid 935514360
logging 11 cells for oeid 990681014
logging 184 cells for oeid 856938751
logging 6 cells for oeid 930996080
logging 7 cells for oeid 944115820
logging 87 cells for oeid 1047025008
logging 167 cells for oeid 

logging 84 cells for oeid 989610990
logging 98 cells for oeid 880961028
logging 128 cells for oeid 1059768347
logging 8 cells for oeid 1048483613
logging 30 cells for oeid 882551939
logging 112 cells for oeid 885146785
logging 26 cells for oeid 882968564
logging 13 cells for oeid 886587170
logging 9 cells for oeid 887386979
logging 10 cells for oeid 881949070
logging 73 cells for oeid 886003525
logging 23 cells for oeid 940352989
logging 141 cells for oeid 930785836
logging 534 cells for oeid 973927944
logging 18 cells for oeid 887386974
logging 20 cells for oeid 881949077
logging 7 cells for oeid 886587164
logging 120 cells for oeid 940852112
logging 19 cells for oeid 886587166
logging 15 cells for oeid 885146237
logging 14 cells for oeid 882551943
logging 18 cells for oeid 887386991
logging 15 cells for oeid 943134119
logging 123 cells for oeid 882551960
logging 7 cells for oeid 891108763
logging 254 cells for oeid 892799212
logging 27 cells for oeid 885146239
logging 150 cells for o

logging 11 cells for oeid 981178529
logging 194 cells for oeid 915150256
logging 7 cells for oeid 984914594
logging 10 cells for oeid 986518865
logging 8 cells for oeid 919418501
logging 17 cells for oeid 984914599
logging 155 cells for oeid 962471847
logging 19 cells for oeid 986518863
logging 143 cells for oeid 836258936
logging 3 cells for oeid 917820379
logging 23 cells for oeid 920288853
logging 46 cells for oeid 974358977
logging 15 cells for oeid 917820375
logging 21 cells for oeid 965224600
logging 57 cells for oeid 992620393
logging 137 cells for oeid 903485705
logging 189 cells for oeid 911146705
logging 17 cells for oeid 921024197
logging 80 cells for oeid 904352694
logging 9 cells for oeid 986518847
logging 23 cells for oeid 917820370
logging 11 cells for oeid 920288851
logging 21 cells for oeid 967008454
logging 20 cells for oeid 986518858
logging 10 cells for oeid 921024187
logging 17 cells for oeid 920288845
logging 120 cells for oeid 964422360
logging 24 cells for oeid 

In [47]:
cell_table

,_id,ophys_experiment_id,ophys_session_id,cell_specimen_id,cell_roi_id,height,mask_image_plane,max_correction_down,max_correction_left,max_correction_right,max_correction_up,roi_mask,valid_roi,width,x,y,entry_time_utc
0,60a2a223f21013bd3cacb0b7,853988454,853416532,1086548282,1080852133,20,0,4.0,3.0,7.0,2.0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,18,186,481,2021-05-17 17:05:23.841769
1,60a2a224f21013bd3cacb0b8,853988454,853416532,1086552401,1080852152,22,0,4.0,3.0,7.0,2.0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,16,132,400,2021-05-17 17:05:24.228770
2,60a2a224f21013bd3cacb0b9,853988454,853416532,1086551752,1080852165,25,0,4.0,3.0,7.0,2.0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,21,233,410,2021-05-17 17:05:24.581785
3,60a2a224f21013bd3cacb0ba,853988454,853416532,1086547687,1080852178,15,0,4.0,3.0,7.0,2.0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,19,282,379,2021-05-17 17:05:24.934078
4,60a2a225f21013bd3cacb0bb,853988454,853416532,1086547776,1080852186,17,0,4.0,3.0,7.0,2.0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,17,108,394,2021-05-17 17:05:25.282879
5,60a2a225f21013bd3cacb0bc,853988454,853416532,1086551518,1080852203,17,0,4.0,3.0,7.0,2.0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,24,130,472,2021-05-17 17:05:25.639308
6,60a2a225f21013bd3cacb0bd,853988454,853416532,1086551191,1080852253,26,0,4.0,3.0,7.0,2.0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,17,479,283,2021-05-17 17:05:25.980099
7,60a2a225f21013bd3cacb0be,853988454,853416532,1086549154,1080852264,18,0,4.0,3.0,7.0,2.0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,22,199,280,2021-05-17 17:05:26.348696
8,60a2a226f21013bd3cacb0bf,853988454,853416532,1086548606,1080852290,24,0,4.0,3.0,7.0,2.0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,24,154,243,2021-05-17 17:05:26.758622
9,60a2a226f21013bd3cacb0c0,853988454,853416532,1086548818,1080852297,18,0,4.0,3.0,7.0,2.0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,23,322,228,2021-05-17 17:05:27.114400
